## Section 1: Business Understanding

#### <u>Question 1</u>: What is the distribution of property types in the Seattle Airbnb dataset?
#### <u>Question 2</u>: What is the set of possible amenities and what does the distribution look like?
#### <u>Question 3</u>: How do amenities correlate with the average guest feedback score for a listing?
#### <u>Question 4</u>: Can we build a predictive model for review scores?


In [339]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import helper as t
pd.options.mode.chained_assignment = None  # default='warn'

from IPython import display
%matplotlib inline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import seaborn as sns

pd.set_option('display.max_columns',85)
pd.set_option('display.max_rows',154)

df = pd.read_csv('./listings.csv')

#### https://www.kaggle.com/airbnb/seattle

## Section 2: Data Understanding (see data-visualization.ipynb) 

In [340]:
# Create new dataframe with columns of interest for the analysis
df1 = df[['host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'property_type', 'room_type','host_response_time',
       'accommodates', 'bathrooms', 'bedrooms', 'beds', 'bed_type',
       'amenities','guests_included','cancellation_policy',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location','reviews_per_month',
       'review_scores_value', 'instant_bookable','price']]

## Section 3: Data Preparation

In [341]:
def convert_to_float(input_string):
    '''
    Remove extraneous characters from string data types and convert to numerical
    
    INPUT 
        input_string - a string containing value to convert to numeric 
        
    OUTPUT
        result - numeric (float) value to 2 decimal places
    '''
    result = str(input_string).replace("$","").replace(",","").replace("%","")
    return round(float(result),2)

def bool_to_int(input_string):
    '''
    Convert boolean data types to integer
    
    INPUT 
        bool_string - a string containing boolean types 't' or 'f' 
        
    OUTPUT
        result - integers 1,0 or None
    '''
    if input_string == 't':
        result = 1
    elif input_string == 'f':
        result = 0
    else: 
        result = None
    return result
    

In [342]:
# List of columns containing string data to convert to float
column_list = ['price','host_response_rate','host_acceptance_rate']

# List of columns containing boolean data to convert to integer
bool_list = ['host_is_superhost',
             'instant_bookable']

for col in column_list:
    df1[col] = df1[col].apply(convert_to_float)
    
for col in bool_list:
    df1[col] = df1[col].apply(bool_to_int)
    


In [343]:
df1.describe()

,host_response_rate,host_acceptance_rate,host_is_superhost,accommodates,bathrooms,bedrooms,beds,guests_included,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,reviews_per_month,review_scores_value,instant_bookable,price
count,3295.000000,3045.000000,3816.000000,3818.000000,3802.000000,3812.000000,3817.000000,3818.000000,3171.000000,3160.000000,3165.000000,3160.000000,3167.000000,3163.000000,3191.000000,3162.000000,3818.000000,3818.000000
mean,94.886798,99.967159,0.203878,3.349398,1.259469,1.307712,1.735394,1.672603,94.539262,9.636392,9.556398,9.786709,9.809599,9.608916,2.078919,9.452245,0.154793,127.976166
std,11.866705,1.812201,0.402932,1.977599,0.590369,0.883395,1.139480,1.311040,6.606083,0.698031,0.797274,0.595499,0.568211,0.629053,1.822348,0.750259,0.361755,90.250022
min,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,20.000000,2.000000,3.000000,2.000000,2.000000,4.000000,0.020000,2.000000,0.000000,20.000000
25%,98.000000,100.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,0.695000,9.000000,0.000000,75.000000
50%,100.000000,100.000000,0.000000,3.000000,1.000000,1.000000,1.000000,1.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.540000,10.000000,0.000000,100.000000
75%,100.000000,100.000000,0.000000,4.000000,1.000000,2.000000,2.000000,2.000000,99.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,10.000000,0.000000,150.000000
max,100.000000,100.000000,1.000000,16.000000,8.000000,7.000000,15.000000,15.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,12.150000,10.000000,1.000000,1000.000000


In [344]:
df1.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,property_type,room_type,host_response_time,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,guests_included,cancellation_policy,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,reviews_per_month,review_scores_value,instant_bookable,price
0,96.0,100.0,0.0,Apartment,Entire home/apt,within a few hours,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",2,moderate,95.0,10.0,10.0,10.0,10.0,9.0,4.07,10.0,0,85.0
1,98.0,100.0,1.0,Apartment,Entire home/apt,within an hour,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",1,strict,96.0,10.0,10.0,10.0,10.0,10.0,1.48,10.0,0,150.0
2,67.0,100.0,0.0,House,Entire home/apt,within a few hours,11,4.5,5.0,7.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",10,strict,97.0,10.0,10.0,10.0,10.0,10.0,1.15,10.0,0,975.0
3,NaN,NaN,0.0,Apartment,Entire home/apt,NaN,3,1.0,0.0,2.0,Real Bed,"{Internet,""Wireless Internet"",Kitchen,""Indoor ...",1,flexible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,100.0
4,100.0,NaN,0.0,House,Entire home/apt,within an hour,6,2.0,3.0,3.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",6,strict,92.0,9.0,9.0,10.0,10.0,9.0,0.89,9.0,0,450.0


In [345]:
# Approach: parse out amenities column and create individual columns for each amenity

def clean(amenities):
    '''
    Clean up amenities column and return a list
    
    INPUT 
        amenities - a string containing a comma separated list of amenities
        
    OUTPUT
        result - a list of amenities
    '''
    amenities = amenities.replace('"','').replace("{",'').replace("}","")
    result = [word for word in amenities.split(',')]
    return result

def parse_result(input_list):
    '''
    Create single list containing a set of all possible amenities
    
    INPUT 
        input_list - a list of lists of amenities
        
    OUTPUT
        amenities_list - a list containing the set of all possible amenities
    '''
    amenities_list = []
    for i in input_list:
        for j in i:
            if j:
                amenities_list.append(j)
    return list(set(amenities_list))   

In [346]:
# clean up amenities column
df1['amenities'] = df1['amenities'].apply(clean)

In [347]:
# create list of lists of amenities
all_amenities_list = list(df1['amenities'])

In [348]:
# generate set of all possible amenities
all_possible = parse_result(all_amenities_list)

### Create Dummy Variables for Amenities 

In [349]:
# Create new dataframe with amenities as 'dummy variables' in columns
df2 = df1.join(df1.pop('amenities').str.join('|').str.get_dummies())

In [350]:
df2.head()

,host_response_rate,host_acceptance_rate,host_is_superhost,property_type,room_type,host_response_time,accommodates,bathrooms,bedrooms,beds,bed_type,guests_included,cancellation_policy,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,reviews_per_month,review_scores_value,instant_bookable,price,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,Dryer,Elevator in Building,Essentials,Family/Kid Friendly,Fire Extinguisher,First Aid Kit,Free Parking on Premises,Gym,Hair Dryer,Hangers,Heating,Hot Tub,Indoor Fireplace,Internet,Iron,Kitchen,Laptop Friendly Workspace,Lock on Bedroom Door,Other pet(s),Pets Allowed,Pets live on this property,Pool,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
0,96.0,100.0,0.0,Apartment,Entire home/apt,within a few hours,4,1.0,1.0,1.0,Real Bed,2,moderate,95.0,10.0,10.0,10.0,10.0,9.0,4.07,10.0,0,85.0,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1
1,98.0,100.0,1.0,Apartment,Entire home/apt,within an hour,4,1.0,1.0,1.0,Real Bed,1,strict,96.0,10.0,10.0,10.0,10.0,10.0,1.48,10.0,0,150.0,0,0,0,1,0,1,0,0,0,1,0,1,1,1,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,1,1,0,0,1
2,67.0,100.0,0.0,House,Entire home/apt,within a few hours,11,4.5,5.0,7.0,Real Bed,10,strict,97.0,10.0,10.0,10.0,10.0,10.0,1.15,10.0,0,975.0,0,1,0,0,1,1,1,1,0,1,0,1,1,0,0,1,0,0,0,1,1,1,1,0,1,0,0,0,1,1,0,0,1,1,0,0,1,1,0,0,1
3,NaN,NaN,0.0,Apartment,Entire home/apt,NaN,3,1.0,0.0,2.0,Real Bed,1,flexible,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,100.0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,1
4,100.0,NaN,0.0,House,Entire home/apt,within an hour,6,2.0,3.0,3.0,Real Bed,6,strict,92.0,9.0,9.0,10.0,10.0,9.0,0.89,9.0,0,450.0,0,0,0,0,1,1,0,0,0,0,0,1,1,1,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1


## Section 4: Fit Linear Model

In [351]:
# Check for missing values in columns
nans = (df2.isnull().sum())

In [352]:
# Missing values sorted by column
nans.sort_values(ascending=False).nlargest(18)

host_acceptance_rate           773
review_scores_checkin          658
review_scores_accuracy         658
review_scores_value            656
review_scores_location         655
review_scores_cleanliness      653
review_scores_communication    651
review_scores_rating           647
reviews_per_month              627
host_response_rate             523
host_response_time             523
bathrooms                       16
bedrooms                         6
host_is_superhost                2
beds                             1
property_type                    1
Washer                           0
Iron                             0
dtype: int64

### Remove outliers


In [353]:
# List of property types to drop from model
drop_list = ['Camper/RV','Boat','Tent','Dorm','Treehouse','Other']

# Filter outliers from price, number of bedrooms, accomodation number, and number of beds
filtr1 = (df2['price'] <= 999) & (df2['price'] >= 49) & (
          df2['bathrooms'] > 0) & (df2['bedrooms'] > 0) & (df2['accommodates'] < 12) & (
          df2['beds'] < 10)

# Filter out property types from drop_list
filtr2 = ~df2['property_type'].isin(drop_list)
df_fit = df2[filtr1 & filtr2]

In [354]:
# Create copy of dataframe to fit model
df_fit = df2.copy()

### Drop rows from key columns with null values
### Impute values for remaining null values

In [355]:
# Columns to drop rows with missing values
col_list = ['review_scores_rating','review_scores_accuracy','review_scores_cleanliness',
            'review_scores_communication','review_scores_checkin','host_is_superhost','host_response_time',
            'review_scores_location','review_scores_value','host_response_rate','host_acceptance_rate',
            'reviews_per_month']

# Columns to impute missing values with mode
# We can justify imputing values for these variables since they are constrained to a relatively small range
# and/or have a lower correlation with review scores
mode_list = ['bathrooms','bedrooms',
             'property_type','beds','instant_bookable']

# Drop rows from col_list
df_fit.dropna(subset=col_list,axis=0,inplace=True)  

# Impute values in mode_list
for col in mode_list:
    df_fit[col].fillna((df_fit[col].mode()[0]), inplace=True)

In [356]:
# Check for any remaining null values
df_fit.isnull().sum().sum()

0

In [357]:
df_fit.describe()

,host_response_rate,host_acceptance_rate,host_is_superhost,accommodates,bathrooms,bedrooms,beds,guests_included,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,reviews_per_month,review_scores_value,instant_bookable,price,24-Hour Check-in,Air Conditioning,Breakfast,Buzzer/Wireless Intercom,Cable TV,Carbon Monoxide Detector,Cat(s),Dog(s),Doorman,Dryer,Elevator in Building,Essentials,Family/Kid Friendly,Fire Extinguisher,First Aid Kit,Free Parking on Premises,Gym,Hair Dryer,Hangers,Heating,Hot Tub,Indoor Fireplace,Internet,Iron,Kitchen,Laptop Friendly Workspace,Lock on Bedroom Door,Other pet(s),Pets Allowed,Pets live on this property,Pool,Safety Card,Shampoo,Smoke Detector,Smoking Allowed,Suitable for Events,TV,Washer,Washer / Dryer,Wheelchair Accessible,Wireless Internet
count,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000,2673.000000
mean,95.331463,99.962589,0.261504,3.386457,1.247288,1.271231,1.743734,1.709689,94.552189,9.638234,9.569398,9.780022,9.804714,9.616910,2.322346,9.460531,0.188178,120.829405,0.167228,0.185559,0.078938,0.147774,0.388328,0.682005,0.099140,0.134680,0.023195,0.786008,0.206884,0.873550,0.534231,0.598204,0.467639,0.578376,0.118593,0.218107,0.228956,0.965956,0.078189,0.219978,0.756079,0.209502,0.897493,0.196409,0.019454,0.013094,0.126824,0.231949,0.047886,0.209502,0.739993,0.878788,0.019454,0.060606,0.681257,0.781893,0.000374,0.082679,0.973438
std,10.889798,1.934196,0.439536,1.994546,0.588863,0.853480,1.161936,1.304173,6.459193,0.692986,0.767846,0.608895,0.575205,0.618368,1.865418,0.735154,0.390927,81.443053,0.373249,0.388823,0.269692,0.354942,0.487461,0.465785,0.298905,0.341445,0.150550,0.410198,0.405147,0.332418,0.498920,0.490353,0.499045,0.493911,0.323370,0.413038,0.420240,0.181377,0.268519,0.414309,0.429526,0.407030,0.303370,0.397355,0.138139,0.113698,0.332838,0.422156,0.213566,0.407030,0.438720,0.326435,0.138139,0.238651,0.466076,0.413038,0.019342,0.275447,0.160829
min,17.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,20.000000,2.000000,4.000000,2.000000,2.000000,4.000000,0.030000,2.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,98.000000,100.000000,0.000000,2.000000,1.000000,1.000000,1.000000,1.000000,93.000000,9.000000,9.000000,10.000000,10.000000,9.000000,0.890000,9.000000,0.000000,70.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
50%,100.000000,100.000000,0.000000,3.000000,1.000000,1.000000,1.000000,1.000000,96.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.860000,10.000000,0.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0

In [358]:
# Select columns with categorical data types
df_fit.select_dtypes(include='object').head()

,property_type,room_type,host_response_time,bed_type,cancellation_policy
0,Apartment,Entire home/apt,within a few hours,Real Bed,moderate
1,Apartment,Entire home/apt,within an hour,Real Bed,strict
2,House,Entire home/apt,within a few hours,Real Bed,strict
6,House,Private room,within an hour,Real Bed,moderate
7,Cabin,Private room,within an hour,Real Bed,strict


In [359]:
# Create copy of the dataframe
df_model = df_fit.copy()

In [360]:
cat_df = df_fit.select_dtypes(include='object') 
#Create a copy of the dataframe
cat_df_copy = cat_df.copy()
#Pull a list of the column names of the categorical variables
cat_cols_lst = list(cat_df.columns)

def create_dummy_df(df, cat_cols, dummy_na):
    '''
    Create dummy variables for categorical columns
    
    INPUT 
        df - input dataframe
        cat_cols - list of categorical columns to convert to dummies
        dummy_na - switch to convert null values to dummies (True/False)
        
    OUTPUT
        df - dataframe with dummy variables in columns
    '''
    for col in cat_cols:
        try:
            # for each cat add dummy var, drop original column
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', drop_first=True, 
                            dummy_na=dummy_na)], axis=1)
        except:
            print('error')
            continue
    return df

In [361]:
#cat_cols_lst

In [362]:
def fit_linear_mod(df, response_col, cat_cols, dummy_na, test_size=.20, rand_state=45):

    #Dummy categorical variables
    df = create_dummy_df(df, cat_cols, dummy_na)
    dfmod = df
    
    #Split into explanatory and response variables
    X = df.drop(response_col, axis=1)
    y = df[response_col]

    #Split into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X_train, y_train) #Fit

    #Predict using model
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)

    #Score using model
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    mean_abs_err = mean_absolute_error(y_test, y_test_preds)

    return dfmod, mean_abs_err, test_score, train_score, lm_model, X_train, X_test, y_train, y_test

In [363]:
#Fit Model

dfmodel, err, test_score, train_score, lm_model, X_train, X_test, y_train, y_test = fit_linear_mod(
                                      df_model,'review_scores_rating', cat_cols_lst, dummy_na=False)

## Section 5: Model Evaluation

In [364]:
print('Mean Absolute Error:', round(err,2))
print("Rsquared score-training data: ", round(train_score,4))
print("Rsquared score-test data: ", round(test_score,4))

Mean Absolute Error: 2.53
Rsquared score-training data:  0.6924
Rsquared score-test data:  0.6564


## Section 6: Model Results

Based on the results from Section 5, we can conclude that the chosen linear model is reasonably well-suited to estimating review scores based on available data. The r-squared result shows us that approximately 66 percent of the variance in the review scores are accounted for by the model.